In [1]:
import pandas as pd
import os
import math

In [ ]:
# import pandas as pd

#     class StudentsDF(pd.DataFrame):
#         SCORES = 'scores'
#         STUDENT_NAME = 'name'

#         @property
#         def _constructor(self):
#             return StudentsDF

#     x = StudentsDF(data=dict(name=['Alice', 'Bob'], scores=[60, 50]), index=[100, 200])
#     type(x)  # __main__.StudentData

In [10]:
class FTESeasonDF(pd.DataFrame):
     
    def mvp(self):
        self.dropna(subset = ['score1'], inplace = True)
        season = '-'.join([self.iloc[0]['date'][0:4],self.iloc[-1]['date'][0:4]])
        home = self.groupby('team1').apply(
            lambda x: (x['score1'] > x['score2']).sum() * 3 + (x['score1'] == x['score2']).sum()).reset_index(name='points')
        pt = pd.pivot_table(self, values = ['score1', 'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1','adj_score2'], 
                            index = 'team1', aggfunc=sum)
        home = home.merge(pt.reset_index(), on = 'team1')
        pt = pd.pivot_table(self, values = 'date', index = 'team1', aggfunc=pd.Series.nunique)
        home = home.merge(pt.reset_index(), on = 'team1')
        cols = {'date':'played', 'team1':'team', 'score1':'scored', 'score2':'conceded', 'xg1':'xg_scored',
                'xg2':'xg_conceded', 'nsxg1':'nsxg_scored', 'nsxg2':'nsxg_conceded', 'adj_score1':'adj_goals_scored',
                'adj_score2':'adj_goals_conceded'}
        home = home.rename(columns = cols)
        away = self.groupby('team2').apply(
            lambda x: (x['score2'] > x['score1']).sum() * 3 + (x['score1'] == x['score2']).sum()).reset_index(name='points')
        pt = pd.pivot_table(self, values = ['score1', 'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1','adj_score2'], 
                            index = 'team2', aggfunc=sum)
        away = away.merge(pt.reset_index(), on = 'team2')
        pt = pd.pivot_table(self, values = 'date', index = 'team2', aggfunc=pd.Series.nunique)
        away = away.merge(pt.reset_index(), on = 'team2')
        cols = {'date':'played', 'team2':'team', 'score2':'scored', 'score1':'conceded', 'xg2':'xg_scored',
                'xg1':'xg_conceded', 'nsxg2':'nsxg_scored', 'nsxg1':'nsxg_conceded', 'adj_score2':'adj_goals_scored',
                'adj_score1':'adj_goals_conceded'}
        away = away.rename(columns = cols)
        t = pd.concat([home, away], ignore_index=True)
        t = t.groupby('team').sum()
        t['goal_difference'] = t['scored'] - t['conceded']
        t['season'] = season
        return t
    
    @property
    def _constructor(self):
        return FTESeasonDF


In [3]:
def fte_last_season(path, league_dict):
    df = pd.read_csv(path)
    for key in league_dict:
        q = ''.join(['league_id == ', league_dict[key]])
        league = df.query(q).reset_index()
        num_teams = len(league.loc[:50, 'team1'].unique())
        i = math.floor(league.shape[0]/(num_teams * (num_teams - 1))) - 1
        return league.loc[((num_teams * (num_teams - 1)) * i):,:]

In [4]:
path = os.path.expanduser('~/Downloads/spi_matches.csv')
league_dict = {'england':'2411'}
# league_dict = {'spain':'1869'}
# league_dict = {'italy':'1854'}
# league_dict = {'germany':'1845'} #, '
# league_dict = {'france':'1843'}
df = fte_last_season(path, league_dict)

In [11]:
df = FTESeasonDF()

In [12]:
df.mvp()

KeyError: ['score1']

In [9]:
type(df)

__main__.FTESeasonDF